Adapted from https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.neural_network = nn.Sequential(
            
            # Layer 1
            nn.Conv2d(1,6,5),
            nn.Tanh(),
            nn.AvgPool2d(2, 2),
            nn.Tanh(),
            
            # Layer 2
            nn.Conv2d(6, 16, 5),
            nn.Tanh(),
            nn.AvgPool2d(2, 2),
            nn.Tanh(),
            
            # Layer 3
            nn.Conv2d(16, 120, 5),
            nn.Tanh(),
            
            # FC
            nn.Flatten(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10),
        )

    def forward(self, x):
        return self.neural_network(x)

In [3]:
transform = transforms.Compose(
    [
        transforms.Pad(2),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)

training_set = datasets.MNIST('./data', train=True, transform=transform, download=True)
validation_set = datasets.MNIST('./data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Failed to download (trying next):
<urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>



100%|██████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 18766700.72it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 1440513.89it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1344165.23it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4521844.00it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Training set has 60000 instances
Validation set has 10000 instances


In [4]:
model = LeNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader): 
        (X, y) = data # len(X) = 10
        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0 and batch > 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
model = LeNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_loader, model, loss_fn, optimizer)
    test_loop(validation_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.755508  [ 4000/60000]
loss: 0.257908  [ 8000/60000]
loss: 0.384730  [12000/60000]
loss: 0.091721  [16000/60000]
loss: 0.548649  [20000/60000]
loss: 0.030786  [24000/60000]
loss: 0.062195  [28000/60000]
loss: 0.049746  [32000/60000]
loss: 0.023731  [36000/60000]
loss: 0.024768  [40000/60000]
loss: 0.003847  [44000/60000]
loss: 0.053367  [48000/60000]
loss: 0.815018  [52000/60000]
loss: 0.033763  [56000/60000]
Test Error: 
 Accuracy: 97.3%, Avg loss: 0.095184 

Epoch 2
-------------------------------
loss: 0.098859  [ 4000/60000]
loss: 0.230185  [ 8000/60000]
loss: 0.010226  [12000/60000]
loss: 0.010907  [16000/60000]
loss: 0.006215  [20000/60000]
loss: 0.046111  [24000/60000]
loss: 0.006118  [28000/60000]
loss: 0.004960  [32000/60000]
loss: 0.002621  [36000/60000]
loss: 0.948711  [40000/60000]
loss: 0.032140  [44000/60000]
loss: 0.128814  [48000/60000]
loss: 0.001831  [52000/60000]
loss: 1.843318  [56000/60000]
Test Error: 
 Accuracy: 97.7